# Detect Phishing URLs
### Capstone 3 - Preprocessing and Modeling
Michael Garber

#### High-Level Steps
1. Preprocessing
    1. Create dummy/indicator features for categorical variables
    2. Standardize/scale numeric features
    3. Train/Test Split 
2. Modeling
    1. Fit your models with a training dataset
    2. Review model outcomes — Iterate over additional models as needed
    3. Identify the final model that you think is the best model for this project

#### Preprocessing

##### Create dummies

##### Standardize and Scale

##### Train/Test Split

#### Modeling

##### Fit Model

##### Evaluate/compare models

##### Select Best Model